In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#Es un entorno sencillo que nos permite hacer pruebas rapidas con QLearning
env = gym.make('FrozenLake-v0')

In [ ]:
tf.reset_default_graph()

In [ ]:
#Si usamos una red neuronal para predecir el valor de un par (Estado,accion)

#Aquí no hay función de activación por lo que todo el modelo será lineal
#Para un caso simple como este basta, pero para casos mas complejos necesitaremos funciones de activacion como RELU o la sigmoide.
#La predicción será la acción que mayor Q tenga

inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))  #Los pesos serán los parámetros que modificará la NN
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

nextQ = tf.placeholder(shape = [1,4], dtype = tf.float32)
#al usar los errores cuadráticos les damos más importancia a los mayores y menos a las pequeñas desviaciones
loss = tf.reduce_sum(tf.square(nextQ - Qout))

trainer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
updateModel = trainer.minimize(loss)

In [ ]:
init = tf.initializers.global_variables()

gamma = .99     #el discount factor, es decir, lo importante que son los rewards futuros para el valor Q
epsilon = .1    #el epsilon nos sirve para controlar el factor de exploración
num_episodes = 4000

episode_list = []      # la cantidad de pasos que tarda cada episodio
reward_list = []       #la lista de rewards de todos los episodios

with tf.Session() as sess:
    sess.run(init) #inicializamos la sesión y las variables globales
    for i in range(num_episodes):
        
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        
        while j < 99:  #99 es el numero maximo de pasos en un episodio
            j += 1
            
            #hacemos una ejecucion de la red neuronal con los datos del estado y sacamos las predicciones junto con la mejor accion
            a,allQ = sess.run([predict,Qout], feed_dict = {inputs1:np.identity(16)[s:s+1]})
            
            #aqui gestionamos la exploración con el parametro epsilon
            if np.random.rand(1) < epsilon:
                a[0] = env.action_space.sample()
                
            #hacemos un step con la acción elegida y recibimos la señal de recompensa del entorno
            s1,r,d,_ = env.step(a[0])
            
            #Recibimos las predicciones de los valores Q para el proximo estado
            Q1 = sess.run(Qout, feed_dict = {inputs1:np.identity(16)[s1:s1+1]})
            
            #obtenemos la mayor para s+1 para poder actualizar el valor de Q con el gamma por el maximo de s+1
            maxQ1 = np.max(Q1)
            
            targetQ = allQ
            #solo hay 1 elemento en la dim 0 y en la segunda son el valor q para cada accion
            #asi que acutalizamos el valor de targetq basandonos en eso
            targetQ[0,a[0]] = r + gamma*maxQ1
            
            #hacemos aqui la parte de backpropagation actualizando la funcion de coste dado el estado actual y como nextQ le pasamos el targetQ calculado
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                epsilon = 1./((i/50) + 10)
                break
        episode_list.append(j)
        reward_list.append(rAll)
        print("Average score per episode: " + str(sum(reward_list)/num_episodes) + " at iteration:" + str(i))
        

In [ ]:
plt.xscale("log")
plt.plot(reward_list)

In [ ]:
plt.xscale("log")
plt.plot(episode_list)